In [7]:
# import torch 
from torch import nn
import torch
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import kurtosis
from prettytable import PrettyTable
import os
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Cuda Device Available")
    print("Name of the Cuda Device: ", torch.cuda.get_device_name())
    print("GPU Computational Capablity: ", torch.cuda.get_device_capability())

else:
    device = torch.device("cpu")
    
def changeCourtToValue(court):
    if court == 'Hard':
        return 1
    elif court == 'Clay':
        return 2
    elif court == 'Grass':
        return 3
    elif court == 'Carpet':
        return 4
    
def changeHandToValue(hand):
    if hand == 'R':
        return 1
    elif hand == 'U':
        return 0
    elif hand == 'L':
        return -1

    
    
def BuildDataset():
    df = pd.read_csv("data/atp_matches_2015.csv")
    for i in range(2016, 2023):
        newDf = pd.read_csv("data/atp_matches_" + str(i) + ".csv")
        df = pd.concat([df, newDf], join="inner", ignore_index = True)
    
    df = df[['surface', 'winner_hand', 'winner_ht', 'winner_age', 'winner_rank', 'winner_rank_points', 'loser_hand', 'loser_ht', 'loser_age', 'loser_rank', 'loser_rank_points']]
    
    
    avgSwitches = ['winner_ht', 'winner_age', 'loser_ht', 'loser_age']
    for i in avgSwitches:
        mean = df[i].mean()
        df[i] = df[i].fillna(mean)
        
    maxSwitches = ['winner_rank', 'loser_rank']
    for i in maxSwitches:
        Max = df[i].max()
        df[i] = df[i].fillna(Max)
    
    minSwitches = ['winner_rank_points', 'loser_rank_points']
    for i in minSwitches:
        df[i] = df[i].fillna(1)
        
    hands = ['winner_hand', 'loser_hand']
    for i in hands:
        df[i] = df[i].fillna('R')
    
    df['surface'] = df['surface'].apply(changeCourtToValue)
    df['winner_hand'] = df['winner_hand'].apply(changeHandToValue)
    df['loser_hand'] = df['loser_hand'].apply(changeHandToValue)
    df['Player1Wins'] = 1
    
    df.rename(columns = {'winner_hand': 'P1Hand', 'winner_ht': 'P1Height', 
                        'winner_age':'P1Age', 'winner_rank':'P1Rank', 'winner_rank_points': 'P1RankPoint',
                         'loser_hand': 'P2Hand', 'loser_ht': 'P2Height', 
                        'loser_age':'P2Age', 'loser_rank':'P2Rank', 'loser_rank_points': 'P2RankPoint'
                        }, inplace = True)
    
    for i in range(19622):
        if (i%2 == 0):
            df = swapPlayers(df, i)
    return df

def swapPlayers(df, index):
    df.loc[index, 'P1Hand'], df.loc[index, 'P2Hand'] = df.loc[index, 'P2Hand'], df.loc[index, 'P1Hand']
    df.loc[index, 'P1Height'], df.loc[index, 'P2Height'] = df.loc[index, 'P2Height'], df.loc[index, 'P1Height']
    df.loc[index, 'P1Age'], df.loc[index, 'P2Age'] = df.loc[index, 'P2Age'], df.loc[index, 'P1Age']
    df.loc[index, 'P1Rank'], df.loc[index, 'P2Rank'] = df.loc[index, 'P2Rank'], df.loc[index, 'P1Rank']
    df.loc[index, 'P1RankPoint'], df.loc[index, 'P2RankPoint'] = df.loc[index, 'P2RankPoint'], df.loc[index, 'P1RankPoint']
    df.loc[index, 'Player1Wins'] = 0
    return df

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.ff1 = nn.Linear(11, 32)
        self.ff2 = nn.Linear(32, 64)
        self.ff3 = nn.Linear(64, 32)
        self.ff4 = nn.Linear(32, 16)
        self.ff5 = nn.Linear(16, 4)
        self.ff6 = nn.Linear(4, 1)

    def forward(self, x):
        x = F.normalize(x)
        print(x, "\n")
        x = F.relu(self.ff1(x))
        print(x, "\n")
        x = F.relu(self.ff2(x))
        print(x, "\n")
        x = F.relu(self.ff3(x))
        print(x, "\n")
        x = F.relu(self.ff4(x))
        print(x, "\n")
        x = F.relu(self.ff5(x))
        print(x, "\n")
        x = torch.sigmoid(self.ff6(x))
        print(x, "\n")
        return x
    
def Train(model, X_train, y_train, epochs, learningRate, batchSize):
    
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
    
    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    dataLoader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batchSize, shuffle=True)
    
    for epoch in range(epochs):
        for i, (data, label) in enumerate(dataLoader):
         #   data = data
            label = label.type(torch.LongTensor)

            output = model(data)
            output = output.squeeze(1)
            output = output.float()
            label = label.float()
            print(output)
            print(label)
            output = torch.nan_to_num(output, nan=0.5)
            loss = criterion(output, label)
            print(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Need Validation
            
        print("Epoch: " + str(epoch) + " Loss: " + str(loss.item()))

    print('Training Complete')
    
    

def main():
    df = BuildDataset()
    print(df.tail())
    
#main()
df = BuildDataset()
M = NeuralNet()
train, test = train_test_split(df, test_size=0.33)
X_train, y_train = torch.tensor(np.array(train.iloc[:, 0:-1])), torch.tensor(np.array(train.iloc[:, -1]))
X_test, y_test = torch.tensor(np.array(test.iloc[:, 0:-1])), torch.tensor(np.array(test.iloc[:, -1]))
X_train, y_train = X_train.float(), y_train.float()
print(X_train[0])
print(y_train[0])

Train(M, X_train, y_train, 20, 0.001, 3)

Cuda Device Available
Name of the Cuda Device:  NVIDIA GeForce RTX 3060 Laptop GPU
GPU Computational Capablity:  (8, 6)
tensor([1.0000e+00, 1.0000e+00, 1.9300e+02, 2.2973e+01, 2.0000e+01, 1.5850e+03,
        1.0000e+00, 1.8500e+02, 2.9317e+01, 1.1000e+01, 2.4000e+03])
tensor(1.)
tensor([[ 1.2829e-03, -6.4143e-04,  1.2380e-01,  2.2459e-02,  2.2450e-02,
          8.2745e-01,  6.4143e-04,  1.1610e-01,  1.8487e-02,  4.1693e-02,
          5.3239e-01],
        [ 6.7984e-04, -6.7984e-04,  1.2917e-01,  2.1243e-02,  2.9913e-02,
          9.0215e-01,  6.7984e-04,  1.2708e-01,  1.9307e-02,  3.8751e-01,
          3.7391e-02],
        [ 7.6358e-04,  7.6358e-04,  1.4508e-01,  2.5212e-02,  5.5741e-02,
          5.3451e-01, -7.6358e-04,  1.4355e-01,  2.5417e-02,  2.6725e-02,
          8.1703e-01]]) 

tensor([[0.3023, 0.0126, 0.4840, 0.1402, 0.0124, 0.0000, 0.0000, 0.0223, 0.2960,
         0.1696, 0.0000, 0.0702, 0.5808, 0.1046, 0.0000, 0.0792, 0.1336, 0.0000,
         0.2497, 0.0000, 0.0000, 0.0000, 0

tensor([[0.0087, 0.0000, 0.0292, 0.0000, 0.0000, 0.0824, 0.0000, 0.1135, 0.2028,
         0.0819, 0.0000, 0.0485, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2057,
         0.0000, 0.0000, 0.0098, 0.0350, 0.0000, 0.0081, 0.0986, 0.0000, 0.0296,
         0.0000, 0.0000, 0.0712, 0.0000, 0.0825],
        [0.0176, 0.0000, 0.0457, 0.0000, 0.0000, 0.0816, 0.0000, 0.0765, 0.1889,
         0.0826, 0.0000, 0.0692, 0.0000, 0.0000, 0.0000, 0.0000, 0.0009, 0.1757,
         0.0000, 0.0087, 0.0191, 0.0355, 0.0000, 0.0206, 0.0935, 0.0000, 0.0443,
         0.0000, 0.0000, 0.0520, 0.0000, 0.0817],
        [0.0116, 0.0000, 0.0328, 0.0000, 0.0000, 0.0846, 0.0000, 0.0972, 0.1965,
         0.0838, 0.0000, 0.0642, 0.0000, 0.0000, 0.0000, 0.0000, 0.0024, 0.1905,
         0.0000, 0.0030, 0.0139, 0.0370, 0.0000, 0.0148, 0.0970, 0.0000, 0.0307,
         0.0000, 0.0000, 0.0639, 0.0000, 0.0815]], grad_fn=<ReluBackward0>) 

tensor([[0.1029, 0.0000, 0.0964, 0.0000, 0.0000, 0.1399, 0.1046, 0.0685, 0.0000,
         0.0

tensor([[0.1038, 0.0000, 0.0949, 0.0000, 0.0000, 0.1231, 0.1180, 0.0656, 0.0000,
         0.0000, 0.0994, 0.0038, 0.0579, 0.0906, 0.0000, 0.0000],
        [0.1040, 0.0000, 0.0948, 0.0000, 0.0000, 0.1382, 0.1052, 0.0689, 0.0000,
         0.0000, 0.0908, 0.0000, 0.0423, 0.0940, 0.0000, 0.0000],
        [0.1363, 0.0000, 0.0694, 0.0000, 0.0000, 0.1190, 0.1463, 0.0712, 0.0000,
         0.0000, 0.0971, 0.0070, 0.0790, 0.0695, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>) 

tensor([[0.2511, 0.0550, 0.0000, 0.1195],
        [0.2491, 0.0526, 0.0000, 0.1211],
        [0.2472, 0.0598, 0.0000, 0.1254]], grad_fn=<ReluBackward0>) 

tensor([[0.5544],
        [0.5541],
        [0.5552]], grad_fn=<SigmoidBackward0>) 

tensor([0.5544, 0.5541, 0.5552], grad_fn=<SqueezeBackward1>)
tensor([1., 0., 1.])
tensor(0.6620, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[1.8372e-04, 9.1858e-05, 1.7269e-02, 3.0604e-03, 9.1858e-05, 9.9758e-01,
         9.1858e-05, 1.6994e-02, 2.5489e-03, 7.9917e-03, 6.4576e-02]

tensor([[0.3463, 0.0000, 0.4945, 0.1414, 0.0000, 0.0000, 0.0000, 0.0535, 0.2971,
         0.1738, 0.0000, 0.0969, 0.5710, 0.0707, 0.0000, 0.0411, 0.1101, 0.0000,
         0.2037, 0.0000, 0.0000, 0.0000, 0.1561, 0.0408, 0.1764, 0.4356, 0.5381,
         0.0000, 0.3361, 0.4284, 0.0000, 0.0320],
        [0.1757, 0.0402, 0.3852, 0.1529, 0.1244, 0.0000, 0.0000, 0.0446, 0.2776,
         0.1170, 0.0000, 0.0000, 0.5485, 0.1514, 0.0000, 0.1823, 0.2192, 0.0000,
         0.3412, 0.0000, 0.0000, 0.0000, 0.0594, 0.0034, 0.1153, 0.3392, 0.4592,
         0.0000, 0.3083, 0.3976, 0.0000, 0.0187],
        [0.2337, 0.0380, 0.4122, 0.1432, 0.0732, 0.0000, 0.0000, 0.0793, 0.2963,
         0.1433, 0.0000, 0.0000, 0.5605, 0.1063, 0.0000, 0.1254, 0.1818, 0.0000,
         0.2993, 0.0000, 0.0000, 0.0000, 0.1042, 0.0133, 0.1528, 0.3785, 0.4885,
         0.0000, 0.3341, 0.4078, 0.0000, 0.0460]], grad_fn=<ReluBackward0>) 

tensor([[0.0000, 0.0212, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1587, 0.1057,
         0.0

tensor([[0.0000e+00, 2.2386e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7468e-04,
         0.0000e+00, 1.5626e-01, 1.0667e-01, 0.0000e+00, 0.0000e+00, 1.7108e-01,
         2.3225e-01, 0.0000e+00, 1.8014e-01, 0.0000e+00, 1.2131e-01, 7.0172e-02,
         1.7287e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.6126e-02, 4.1346e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 4.6260e-02, 1.1132e-01, 2.1589e-01,
         2.0071e-01, 0.0000e+00, 2.1272e-01, 0.0000e+00, 0.0000e+00, 1.5182e-01,
         0.0000e+00, 1.6135e-01, 4.7893e-02, 1.8598e-01, 0.0000e+00, 4.3958e-02,
         0.0000e+00, 1.3026e-01, 9.5379e-02, 1.0054e-01, 1.5845e-01, 4.9727e-02,
         0.0000e+00, 0.0000e+00, 1.5314e-01, 1.3602e-01, 2.5540e-01, 0.0000e+00,
         7.8204e-02, 3.2350e-01, 1.0091e-01, 1.9080e-01, 0.0000e+00, 3.9011e-02,
         0.0000e+00, 0.0000e+00, 5.0940e-02, 2.5029e-01],
        [0.0000e+00, 4.0596e-02, 0.0000e+00, 0.0000e+00, 4.4632e-03, 1.9862e-02,
         0.0000e+00, 1.2589e-01, 1.0506e-01, 0.0000

...

tensor([0.5435, 0.5433, 0.5440], grad_fn=<SqueezeBackward1>)
tensor([0., 0., 0.])
tensor(0.7844, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([[ 2.8282e-03,  9.4272e-04,  1.7252e-01,  2.7385e-02,  5.2792e-02,
          8.7767e-01,  9.4272e-04,  1.7440e-01,  2.5018e-02,  1.2915e-01,
          3.8557e-01],
        [ 9.0732e-04,  9.0732e-04,  1.6332e-01,  2.1021e-02,  8.8010e-02,
          6.9683e-01,  9.0732e-04,  1.6332e-01,  2.8461e-02,  9.2547e-02,
          6.6598e-01],
        [ 6.6007e-04, -3.3004e-04,  6.2047e-02,  7.5061e-03,  3.3664e-02,
          1.9769e-01,  3.3004e-04,  6.7987e-02,  1.0914e-02,  3.3004e-03,
          9.7526e-01]]) 

tensor([[0.2800, 0.0156, 0.4850, 0.1632, 0.0304, 0.0000, 0.0000, 0.0000, 0.3000,
         0.1455, 0.0000, 0.0826, 0.5912, 0.1357, 0.0000, 0.1253, 0.1376, 0.0000,
         0.2809, 0.0000, 0.0000, 0.0000, 0.0833, 0.0289, 0.1248, 0.4009, 0.5172,
         0.0000, 0.2712, 0.4303, 0.0000, 0.0000],
        [0.3709, 0.0000, 0.5313, 0.1445, 0.0000, 0.0000

tensor([[0.0154, 0.0000, 0.0372, 0.0000, 0.0000, 0.0904, 0.0000, 0.0862, 0.1879,
         0.0860, 0.0000, 0.0738, 0.0000, 0.0000, 0.0000, 0.0000, 0.0051, 0.1805,
         0.0000, 0.0083, 0.0177, 0.0353, 0.0000, 0.0191, 0.0944, 0.0000, 0.0320,
         0.0000, 0.0000, 0.0590, 0.0000, 0.0811],
        [0.0214, 0.0000, 0.0536, 0.0000, 0.0000, 0.0955, 0.0000, 0.0620, 0.1719,
         0.0897, 0.0000, 0.0885, 0.0000, 0.0000, 0.0000, 0.0000, 0.0076, 0.1602,
         0.0000, 0.0191, 0.0295, 0.0334, 0.0000, 0.0208, 0.0845, 0.0000, 0.0390,
         0.0000, 0.0000, 0.0451, 0.0000, 0.0795],
        [0.0111, 0.0000, 0.0279, 0.0000, 0.0000, 0.0850, 0.0000, 0.1105, 0.2009,
         0.0856, 0.0000, 0.0568, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2014,
         0.0000, 0.0000, 0.0100, 0.0332, 0.0000, 0.0116, 0.1016, 0.0000, 0.0271,
         0.0000, 0.0000, 0.0702, 0.0000, 0.0835]], grad_fn=<ReluBackward0>) 

tensor([[0.1023, 0.0000, 0.0979, 0.0000, 0.0000, 0.1258, 0.1055, 0.0628, 0.0000,
         0.0

tensor([[0.0216, 0.0000, 0.0404, 0.0000, 0.0000, 0.0783, 0.0000, 0.1098, 0.1972,
         0.0777, 0.0000, 0.0364, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2090,
         0.0000, 0.0000, 0.0107, 0.0297, 0.0000, 0.0033, 0.0953, 0.0000, 0.0419,
         0.0000, 0.0000, 0.0644, 0.0000, 0.0804],
        [0.0230, 0.0000, 0.0415, 0.0000, 0.0000, 0.0771, 0.0000, 0.0945, 0.1901,
         0.0817, 0.0000, 0.0525, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1921,
         0.0000, 0.0000, 0.0133, 0.0288, 0.0000, 0.0134, 0.0972, 0.0000, 0.0487,
         0.0000, 0.0000, 0.0563, 0.0000, 0.0812],
        [0.0178, 0.0000, 0.0377, 0.0000, 0.0000, 0.0808, 0.0000, 0.0909, 0.1903,
         0.0835, 0.0000, 0.0618, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1868,
         0.0000, 0.0003, 0.0152, 0.0305, 0.0000, 0.0166, 0.0997, 0.0000, 0.0439,
         0.0000, 0.0000, 0.0576, 0.0000, 0.0836]], grad_fn=<ReluBackward0>) 

tensor([[0.1075, 0.0000, 0.0947, 0.0000, 0.0000, 0.1347, 0.1011, 0.0683, 0.0000,
         0.0

tensor([[0.0000e+00, 4.9669e-02, 0.0000e+00, 0.0000e+00, 2.0466e-02, 2.0278e-02,
         0.0000e+00, 1.0465e-01, 1.0125e-01, 0.0000e+00, 0.0000e+00, 1.7618e-01,
         1.5884e-01, 2.3928e-02, 2.1403e-01, 0.0000e+00, 1.2913e-01, 8.4713e-02,
         1.5236e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.0717e-02, 3.8778e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 6.2640e-02, 1.1193e-01, 1.9165e-01,
         2.0444e-01, 0.0000e+00, 1.6009e-01, 0.0000e+00, 0.0000e+00, 1.5302e-01,
         0.0000e+00, 1.3695e-01, 3.1848e-02, 1.8292e-01, 0.0000e+00, 6.1350e-02,
         0.0000e+00, 1.0862e-01, 4.9185e-02, 1.0073e-01, 1.7607e-01, 3.4608e-02,
         0.0000e+00, 1.5010e-02, 1.8916e-01, 1.5306e-01, 2.5236e-01, 0.0000e+00,
         7.1715e-02, 2.9980e-01, 5.1303e-02, 1.8855e-01, 0.0000e+00, 3.8229e-02,
         0.0000e+00, 0.0000e+00, 9.7262e-02, 2.3030e-01],
        [0.0000e+00, 2.1989e-02, 0.0000e+00, 0.0000e+00, 2.8886e-04, 6.0732e-04,
         0.0000e+00, 1.5175e-01, 1.0647e-01, 0.0000

KeyboardInterrupt: 

In [ ]:
df.isna().sum()